In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import librosa
import glob
import pandas as pd
import os
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.models import model_from_json


# TODO: Check number of input features
def get_mfcc_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None
    
    return mfccsscaled


# try loading data
data_loaded = True
X_train = None
y_train = None

try:
    X_train = np.load('../input/saved-input/X_train.npy')
    y_train = np.load('../input/saved-input/y_train.npy')
    print("Input loaded successfully")
except IOError:
    print("Input data has not been saved")
    data_loaded = False


# prepare input data
base_path = "../input/birdsong-recognition"
train_audio_path = base_path + "/train_audio"
file1 = train_audio_path + "/aldfly/XC134874.mp3"
train_audio_list = glob.glob(train_audio_path + '/*/*.mp3')
print(len(train_audio_list))

train_csv = pd.read_csv(base_path + '/train.csv')
train_csv.head()

if not data_loaded:
    labels = []
    features = []

    for path in tqdm(train_audio_list):
        base_file = os.path.basename(path)
        #feature = get_mfcc_features(path)
        #if feature is None:
            #continue
        #features.append(feature)
        label = train_csv.loc[train_csv.filename == base_file, 'ebird_code'].values[0]
        labels.append(label)

    X_train = np.array(features)
    y_train = np.array(labels)
    
    encoder = LabelEncoder()
    y_train = to_categorical(encoder.fit_transform(y_train))

    # save generated data
    np.save('X_train', X_train)
    np.save('y_train', y_train)

print(X_train.shape)
print(y_train.shape)
print(y_train[:2, :])


# build the model
model = Sequential()

model.add(Dense(1024, input_shape=(40,), activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(264, activation='softmax'))

# Display model architecture summary 
model.summary()

adam = Adam(lr=0.001)

model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

# Calculate pre-training accuracy 
score = model.evaluate(X_train, y_train, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

# train the model
history = model.fit(x=X_train, y=y_train, epochs=20, batch_size=32, shuffle=True)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")




# Reference: https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7